### Ppurpose built models dataset preparation

In [ ]:
import json
import os
import shutil

### FIXME

1. Assign a model_name in FIXME 1

    1.1 Assign model type for action_recognition/pose_classification in FIXME 1.1

    1.2 Assign platform for action_recognition in FIXME 1.2
    
    1.3 Assign model input type for action_recognition in FIXME 1.3
1. Choose between default and custom dataset in FIXME 2
1. Assign path of DATA_DIR in FIXME 3
1. Assign Cloud credentials in FIXME 4

In [ ]:
# Define model_name workspaces and other variables
# Available models (#FIXME 1):
# 1. action_recognition - https://docs.nvidia.com/tao/tao-toolkit/text/action_recognition_net.html
# 2. bevfusion - https://docs.nvidia.com/tao/tao-toolkit/text/action_recognition_net.html
# 3. ml_recog - https://docs.nvidia.com/tao/tao-toolkit/text/ml_recog/index.html
# 4. ocdnet - https://docs.nvidia.com/tao/tao-toolkit/text/ocdnet/index.html
# 5. ocrnet - https://docs.nvidia.com/tao/tao-toolkit/text/ocrnet/index.html
# 6. optical_inspection - https://docs.nvidia.com/tao/tao-toolkit/text/optical_inspection/index.html
# 7. pose_classification - https://docs.nvidia.com/tao/tao-toolkit/text/pose_classification/index.html
# 8. pointpillars - https://docs.nvidia.com/tao/tao-toolkit/text/point_cloud/pointpillars.html
# 9. re_identification - https://docs.nvidia.com/tao/tao-toolkit/text/re_identification/index.html
# 10. centerpose - https://docs.nvidia.com/tao/tao-toolkit/text/centerpose/index.html
# 11. visual_changenet_segment - https://docs.nvidia.com/tao/tao-toolkit/text/visual_changenet/index.html
# 12. visual_changenet_classify - https://docs.nvidia.com/tao/tao-toolkit/text/visual_changenet/index.html 

model_name = "bevfusion" # FIXME1 (Add the model name from the above mentioned list)

In [ ]:
if model_name in ("action_recognition","pose_classification"):
    # FIXME1.1 - model_type - string
        # action-recognition: rgb/of/joint;
        # pose-classification: kinetics/nvidia
    model_type = "rgb"

    if model_name == "action_recognition":
        if model_type not in ("rgb","of","joint"):
            raise Exception("Choose one of rgb/of/joint for action recognition model_type")
    elif model_name == "pose_classification":
        if model_type not in ("kinetics","nvidia"):
            raise Exception("Choose one of kinetics/nvidia for pose classification model_type (preferred nvidia)")

    if model_name == "action_recognition":
        platform = "a100" # FIXME1.2 a100/xavier - valid only for model_type that is not rgb
        model_input_type = "3d" # FIXME1.3 3d/2d

### Example dataset source and structure <a class="anchor" id="head-1.1"></a>

**Action Recognition:** We will be using the HMDB51 [dataset](https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database/) for the tutorial. (We choose catch/smile for this tutorial):

**MLRecogNet** We will be using the `Retail Product Checkout Dataset` for the tutorial. Downdload the datsaet from [here](https://www.kaggle.com/datasets/diyer22/retail-product-checkout-dataset) and place it under $DATA_DIR/metric_learning_recognition

**OCDNET**: We will be using the ICDAR2015 dataset for the ocdnet tutorial. Please access the dataset [here](https://rrc.cvc.uab.es/?ch=4&com=tasks) to register and download the data from Task 4.1: Text Localization. Unzip the files to DATA_DIR

**OCRNET**: We will be using the ICDAR15 word recognition dataset for the tutorial. To find more details please visit [here](
https://rrc.cvc.uab.es/?ch=4&com=tasks). Please download the ICDAR15 word recognition train dataset and test_dataset [here](https://rrc.cvc.uab.es/?ch=4&com=downloads) to DATA_DIR.

**Pointpillars:** We will be using the `kitti object detection dataset` for this example. To find more details, please visit [here](http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d)

**Pose Classification:** We will be using the Kinetics dataset from [Deepmind](https://deepmind.com/research/open-source/kinetics) or NVIDIA created dataset. For kinetics based dataset set model_type as `kinetics` and for nvidia based dataset set model_type as `nvidia`

**Re-Identification:** We will be using the [Market-1501](https://zheng-lab.cecs.anu.edu.au/Project/project_reid.html) dataset. Download the dataset [here](https://drive.google.com/file/d/1TwkgQcIa_EgRjVMPSbyEKtcfljqURrzi/view?usp=sharing) and extract it.

**Optical Inspection:** Bring your own dataset according to the format described [here](https://docs.nvidia.com/tao/tao-toolkit/text/data_annotation_format.html#optical-inspection-format). 

**Visual ChangeNet-Classification:** Bring your own dataset according to the format described [here](https://docs.nvidia.com/tao/tao-toolkit/text/data_annotation_format.html#optical-inspection-format). 

**Visual ChangeNet-Segmentation:** We will be using the [Market-1501](https://zheng-lab.cecs.anu.edu.au/Project/project_reid.html) dataset. Download the dataset [here](https://www.dropbox.com/s/18fb5jo0npu5evm/LEVIR-CD256.zip) and extract it. 

**CenterPose:** We will be using [Google Objectron](https://github.com/google-research-datasets/Objectron) dataset. The following script will download and preprocess the dataset the dataset automatically.

In [ ]:
dataset_to_be_used = "default" #FIXME2 #default/custom; default for the dataset used in this tutorial notebook; custom for a different dataset
DATA_DIR = os.path.abspath(model_name) #FIXME3 (set absolute path of the data_directory)
os.environ['DATA_DIR']= DATA_DIR
!mkdir -p $DATA_DIR

### Dataset download and pre-processing <a class="anchor" id="head-1"></a>

In [ ]:
if dataset_to_be_used == "default":
    if model_name == "action_recognition":
        !sudo apt-get update -y && sudo apt-get install unrar -y
        !wget -P $DATA_DIR --no-check-certificate http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
        assert os.path.exists(f"{DATA_DIR}/hmdb51_org.rar")
        !mkdir -p $DATA_DIR/videos && unrar x -o+ $DATA_DIR/hmdb51_org.rar $DATA_DIR/videos
        !mkdir -p $DATA_DIR/raw_data
        !unrar x -o+ $DATA_DIR/videos/catch.rar $DATA_DIR/raw_data
        !unrar x -o+ $DATA_DIR/videos/smile.rar $DATA_DIR/raw_data
        assert os.path.exists(f"{DATA_DIR}/raw_data/catch")
        assert os.path.exists(f"{DATA_DIR}/raw_data/smile")
    elif model_name == "ml_recog":
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset.zip")
        !unzip -uq $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset.zip -d $DATA_DIR/metric_learning_recognition
    elif model_name == "ocdnet":
        assert(os.path.exists(f"{DATA_DIR}/train/img"))
        assert(os.path.exists(f"{DATA_DIR}/train/gt"))
        assert(os.path.exists(f"{DATA_DIR}/test/img"))
        assert(os.path.exists(f"{DATA_DIR}/test/gt"))
    elif model_name == "ocrnet":
        !mkdir -p $DATA_DIR/train && rm -rf $DATA_DIR/train/*
        !mkdir -p $DATA_DIR/test && rm -rf $DATA_DIR/test/*
        !unzip -u $DATA_DIR/ch4_test_word_images_gt.zip -d $DATA_DIR/test
        !cp $DATA_DIR/Challenge4_Test_Task3_GT.txt -d $DATA_DIR/test
        !unzip -u $DATA_DIR/ch4_training_word_images_gt.zip -d $DATA_DIR/train    
        assert os.path.exists(f"{DATA_DIR}/ch4_test_word_images_gt.zip")
        assert os.path.exists(f"{DATA_DIR}/Challenge4_Test_Task3_GT.txt")
        assert os.path.exists(f"{DATA_DIR}/ch4_training_word_images_gt.zip")
    elif model_name in ("optical_inspection", "visual_changenet_classify"):
        assert os.path.exists(f"{DATA_DIR}/train/images")
        assert os.path.exists(f"{DATA_DIR}/train/dataset.csv")
        assert os.path.exists(f"{DATA_DIR}/val/images")
        assert os.path.exists(f"{DATA_DIR}/val/dataset.csv")
        assert os.path.exists(f"{DATA_DIR}/test/images")
        assert os.path.exists(f"{DATA_DIR}/test/dataset.csv")
    elif model_name == "visual_changenet_segment":
        #Download the data
        URL_DATASET = "https://www.dropbox.com/s/18fb5jo0npu5evm/LEVIR-CD256.zip"
        os.environ["URL_DATASET"]=URL_DATASET
        !if [ ! -f $DATA_DIR/LEVIR-CD256.zip ]; then wget $URL_DATASET -O $DATA_DIR/LEVIR-CD-256.zip; else echo "image archive already downloaded"; fi 
        # Check the dataset is present
        !mkdir -p $DATA_DIR
        !if [ ! -f $DATA_DIR/LEVIR-CD-256.zip ]; then echo 'Dataset zip file not found, please download.'; else echo 'Found Dataset zip file.';fi
        # unpack 
        !unzip -u $DATA_DIR/LEVIR-CD-256.zip -d $DATA_DIR
        assert os.path.exists(f"{DATA_DIR}/LEVIR-CD256/A")
        assert os.path.exists(f"{DATA_DIR}/LEVIR-CD256/B")
        assert os.path.exists(f"{DATA_DIR}/LEVIR-CD256/label")
        assert os.path.exists(f"{DATA_DIR}/LEVIR-CD256/list/train.txt")
        assert os.path.exists(f"{DATA_DIR}/LEVIR-CD256/list/val.txt")
        assert os.path.exists(f"{DATA_DIR}/LEVIR-CD256/list/test.txt")
    elif model_name in ("pointpillars", "bevfusion"):
        !unzip -u $DATA_DIR/data_object_image_2.zip -d $DATA_DIR
        !unzip -u $DATA_DIR/data_object_label_2.zip -d $DATA_DIR
        !unzip -u $DATA_DIR/data_object_velodyne.zip -d $DATA_DIR
        !unzip -u $DATA_DIR/data_object_calib.zip -d $DATA_DIR

        if model_name == "bevfusion":
            !mkdir -p $DATA_DIR/ImageSets
            !if [ ! -f $DATA_DIR//ImageSets/test.txt ]; then wget https://raw.githubusercontent.com/traveller59/second.pytorch/master/second/data/ImageSets/test.txt --no-check-certificate --content-disposition -O $DATA_DIR//ImageSets/test.txt; else echo "test.txt archive already downloaded"; fi 
            !if [ ! -f $DATA_DIR//ImageSets/train.txt ]; then wget https://raw.githubusercontent.com/traveller59/second.pytorch/master/second/data/ImageSets/train.txt --no-check-certificate --content-disposition -O $DATA_DIR/ImageSets/train.txt; else echo "train.txt archive already downloaded"; fi 
            !if [ ! -f $DATA_DIR//ImageSets/val.txt ]; then wget https://raw.githubusercontent.com/traveller59/second.pytorch/master/second/data/ImageSets/val.txt --no-check-certificate --content-disposition -O $DATA_DIR/ImageSets/val.txt; else echo "val.txt archive already downloaded"; fi 
            !if [ ! -f $DATA_DIR//ImageSets/trainval.txt ]; then wget https://raw.githubusercontent.com/traveller59/second.pytorch/master/second/data/ImageSets/trainval.txt --no-check-certificate --content-disposition -O $DATA_DIR/ImageSets/trainval.txt; else echo "trainval.txt archive already downloaded"; fi

        assert os.path.exists(f"{DATA_DIR}/training/image_2")
        assert os.path.exists(f"{DATA_DIR}/training/label_2")
        assert os.path.exists(f"{DATA_DIR}/training/velodyne")
        assert os.path.exists(f"{DATA_DIR}/training/calib")
    elif model_name == "pose_classification":
        !pip3 install -U gdown
        if model_type == "kinetics":
            !gdown https://drive.google.com/uc?id=1dmzCRQsFXJ18BlXj1G9sbDnsclXIdDdR -O $DATA_DIR/st-gcn-processed-data.zip
            !unzip $DATA_DIR/st-gcn-processed-data.zip -d $DATA_DIR
            !mv $DATA_DIR/data/Kinetics/kinetics-skeleton $DATA_DIR/kinetics
            !rm -r $DATA_DIR/data
            !rm $DATA_DIR/st-gcn-processed-data.zip
            assert os.path.exists(f"{DATA_DIR}/kinetics")
        elif model_type == "nvidia":
            !gdown https://drive.google.com/uc?id=1GhSt53-7MlFfauEZ2YkuzOaZVNIGo_c- -O $DATA_DIR/data_3dbp_nvidia.zip
            !mkdir -p $DATA_DIR/nvidia
            !unzip $DATA_DIR/data_3dbp_nvidia.zip -d $DATA_DIR/nvidia
            !rm $DATA_DIR/data_3dbp_nvidia.zip
            assert os.path.exists(f"{DATA_DIR}/nvidia")
            assert os.path.exists(f"{DATA_DIR}/{model_type}/train_data.npy") and os.path.exists(f"{DATA_DIR}/{model_type}/train_label.pkl") and os.path.exists(f"{DATA_DIR}/{model_type}/val_data.npy") and os.path.exists(f"{DATA_DIR}/{model_type}/val_label.pkl")
    elif model_name == "re_identification":
        !pip3 install -U gdown
        !gdown https://drive.google.com/uc?id=0B8-rUzbwVRk0c054eEozWG9COHM -O $DATA_DIR/market1501.zip
        !unzip -u $DATA_DIR/market1501.zip -d $DATA_DIR
        !rm -rf $DATA_DIR/market1501
        !mv $DATA_DIR/Market-1501-v15.09.15 $DATA_DIR/market1501
        !rm $DATA_DIR/market1501.zip
        assert os.path.exists(f"{DATA_DIR}/market1501")

In [ ]:
if dataset_to_be_used == "default":
    if model_name == "action_recognition":
        !python3 -m pip install opencv-python numpy
        # For rgb action recognition
        !if [ -d tao_toolkit_recipes ]; then rm -rf tao_toolkit_recipes; fi
        !git clone https://github.com/NVIDIA-AI-IOT/tao_toolkit_recipes
        assert os.path.exists("tao_toolkit_recipes")
        !cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && bash ./preprocess_HMDB_RGB.sh $DATA_DIR/raw_data $DATA_DIR/processed_data 

        # For optical flow, comment the above 3 lines and uncomment the below (Note: for generating optical flow, a Turing or Ampere above GPU is needed.)
        #!echo <passwd> | sudo -S apt install -y libfreeimage-dev
        #!cp action_recognition/AppOFCuda tao_toolkit_recipes/tao_action_recognition/data_generation/
        #!cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && bash ./preprocess_HMDB.sh $DATA_DIR/raw_data $DATA_DIR/processed_data

        # download the split files and unrar
        !wget -P $DATA_DIR --no-check-certificate http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar
        assert os.path.exists(f"{DATA_DIR}/test_train_splits.rar")
        !mkdir -p $DATA_DIR/splits && unrar x -o+ $DATA_DIR/test_train_splits.rar $DATA_DIR/splits
        assert os.path.exists(f"{DATA_DIR}/splits")
        # run split_HMDB to generate training split
        !if [ -d $DATA_DIR/train ]; then rm -rf $DATA_DIR/train $DATA_DIR/test; fi
        !cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && python3 ./split_dataset.py $DATA_DIR/processed_data $DATA_DIR/splits/testTrainMulti_7030_splits $DATA_DIR/train  $DATA_DIR/test
        assert os.path.exists(f'{DATA_DIR}/train')
        assert os.path.exists(f'{DATA_DIR}/test')

        if os.path.exists("tao_toolkit_recipes"):
            shutil.rmtree("tao_toolkit_recipes")

        assert not os.path.exists("tao_toolkit_recipes")

    elif model_name == "ml_recog":
        # crops images from detection set and form a classification set
        # splits to reference/train/val/test set
        !sudo apt-get update && sudo apt-get install gcc -y
        !python3 -m pip install opencv-python numpy pycocotools tqdm
        !python3 metric_learning_recognition/process_retail_product_checkout_dataset.py
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/train")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/train")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/test")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/test")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/val")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/val")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/reference")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/reference")

    elif model_name == "ocrnet":
        orig_train_gt_file=os.path.join(os.getenv("DATA_DIR"), "train", "gt.txt")
        processed_train_gt_file=os.path.join(os.getenv("DATA_DIR"), "train", "gt_new.txt")
        orig_test_gt_file=os.path.join(os.getenv("DATA_DIR"), "test", "Challenge4_Test_Task3_GT.txt")
        processed_test_gt_file=os.path.join(os.getenv("DATA_DIR"), "test", "gt_new.txt")
        !python3 ocrnet/preprocess_label.py $orig_train_gt_file $processed_train_gt_file
        !python3 ocrnet/preprocess_label.py $orig_test_gt_file $processed_test_gt_file

    elif model_name == "pointpillars":
        !python3 -m pip install scikit-image numpy
        obtain_subset = True
        if obtain_subset:
            !python pointpillars/obtain_subset.py --source-data-dir=$DATA_DIR/training --out-data-dir=$DATA_DIR/subset_data/training/ --training True --num-images=100
            !python pointpillars/obtain_subset.py --source-data-dir=$DATA_DIR/testing --out-data-dir=$DATA_DIR/subset_data/testing/ --num-images=100
            DATA_DIR += "/subset_data"
            os.environ['DATA_DIR'] += "/subset_data"
        !mkdir -p $DATA_DIR/train/lidar $DATA_DIR/train/label $DATA_DIR/val/lidar $DATA_DIR/val/label

        # Convert labels from Camera coordinate system to LIDAR coordinate system, etc
        !python3 pointpillars/gen_lidar_points.py -p $DATA_DIR/training/velodyne \
                                               -c $DATA_DIR/training/calib    \
                                               -i $DATA_DIR/training/image_2  \
                                               -o $DATA_DIR/train/lidar
        assert os.listdir(f"{DATA_DIR}/train/lidar")
        # Drop DontCare class
        !python3 pointpillars/gen_lidar_labels.py -l $DATA_DIR/training/label_2 \
                                               -c $DATA_DIR/training/calib \
                                               -o $DATA_DIR/train/label
        # train/val split
        !python3 pointpillars/drop_class.py $DATA_DIR/train/label DontCare
        assert os.listdir(f"{DATA_DIR}/train/label")
        # Change the val set id's if you need a different set of validation images
        !python3 pointpillars/kitti_split.py pointpillars/val.txt \
                                          $DATA_DIR/train/lidar \
                                          $DATA_DIR/train/label \
                                          $DATA_DIR/val/lidar \
                                          $DATA_DIR/val/label
        assert os.listdir(f"{DATA_DIR}/val/label")
        assert os.listdir(f"{DATA_DIR}/val/lidar")

    elif model_name == "pose_classification" and model_type == "kinetics":
        !pip3 install numpy
        # select actions
        !python3 pose_classification/select_subset_actions.py
        assert os.path.exists(f"{DATA_DIR}/{model_type}/train_data.npy") and os.path.exists(f"{DATA_DIR}/{model_type}/train_label.pkl") and os.path.exists(f"{DATA_DIR}/{model_type}/val_data.npy") and os.path.exists(f"{DATA_DIR}/{model_type}/val_label.pkl")

    elif model_name == "re_identification":
        #100 is the number of samples to be present in the subset data - you can choose any number <= total samples in the dataset
        !python3 re_identification/obtain_subset_data.py 100
        assert os.path.exists(f"{DATA_DIR}/market1501/sample_train")
        assert os.path.exists(f"{DATA_DIR}/market1501/sample_test")
        assert os.path.exists(f"{DATA_DIR}/market1501/sample_query")

    elif model_name == "centerpose":
        # Select the training categories from: bike, book, bottle, camera, cereal_box, chair, laptop, shoe
        # Please set the "n" to -1 if you want to run the whole dataset training.
        testing_categories = 'bike'
        !pip3 install numpy opencv-python tqdm scipy==1.9.2 tensorflow==2.14.0
        !python3 centerpose/prepare_centerpose_dataset.py \
                                            -c $testing_categories \
                                            -n 100
        assert os.path.exists(f"{DATA_DIR}/{testing_categories}/train")
        assert os.path.exists(f"{DATA_DIR}/{testing_categories}/test")
        assert os.path.exists(f"{DATA_DIR}/{testing_categories}/val")

### Tar the datasets <a class="anchor" id="head-1.3"></a>

In [ ]:
if model_name == "action_recognition":
    !mkdir -p $DATA_DIR/cloud_folders/data/purpose_built_models_action_recognition_train/test
    !tar -C $DATA_DIR -czf $DATA_DIR/cloud_folders/data/purpose_built_models_action_recognition_train/train.tar.gz train
    !tar -C $DATA_DIR -czf $DATA_DIR/cloud_folders/data/purpose_built_models_action_recognition_train/test.tar.gz test
    !tar -C $DATA_DIR/test -czf $DATA_DIR/cloud_folders/data/purpose_built_models_action_recognition_train/test/smile.tar.gz smile
if model_name == "bevfusion":
    !mkdir -p $DATA_DIR/cloud_folders/data/purpose_built_models_action_recognition_train/test
    !tar -C $DATA_DIR -czf $DATA_DIR/cloud_folders/data/purpose_built_models_action_recognition_train/train.tar.gz train
    !tar -C $DATA_DIR -czf $DATA_DIR/cloud_folders/data/purpose_built_models_action_recognition_train/test.tar.gz test
    !tar -C $DATA_DIR/test -czf $DATA_DIR/cloud_folders/data/purpose_built_models_action_recognition_train/test/smile.tar.gz smile
elif model_name == "ml_recog":
    !tar -C $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/ -czf $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/reference.tar.gz reference
    !tar -C $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/ -czf $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/test.tar.gz test
    !tar -C $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/ -czf $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/train.tar.gz train
    !tar -C $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/ -czf $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/val.tar.gz val
    !rm -rf $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/reference $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/test $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/train $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/val

    !tar -C $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/ -czf $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/reference.tar.gz reference
    !tar -C $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/ -czf $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/test.tar.gz test
    !tar -C $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/ -czf $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/train.tar.gz train
    !tar -C $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/ -czf $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/val.tar.gz val
    !rm -rf $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/reference $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/test $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/train $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/val
elif model_name == "ocdnet":
    !mkdir -p $DATA_DIR/cloud_folders/data/purpose_built_models_ocdnet_train/train $DATA_DIR/cloud_folders/data/purpose_built_models_ocdnet_val/test
    !tar -C $DATA_DIR -czf $DATA_DIR/cloud_folders/data/purpose_built_models_ocdnet_train/train.tar.gz train
    !tar -C $DATA_DIR/train -czf $DATA_DIR/cloud_folders/data/purpose_built_models_ocdnet_train/train/img.tar.gz img
    !tar -C $DATA_DIR -czf $DATA_DIR/cloud_folders/data/purpose_built_models_ocdnet_val/test.tar.gz test
    !tar -C $DATA_DIR/test -czf $DATA_DIR/cloud_folders/data/purpose_built_models_ocdnet_val/test/img.tar.gz img
elif model_name == "ocrnet":
    !mkdir -p $DATA_DIR/cloud_folders/data/purpose_built_models_ocrnet_train $DATA_DIR/cloud_folders/data/purpose_built_models_ocrnet_val
    !tar -C $DATA_DIR -czf $DATA_DIR/cloud_folders/data/purpose_built_models_ocrnet_train/train.tar.gz train
    !cp $DATA_DIR/character_list $DATA_DIR/cloud_folders/data/purpose_built_models_ocrnet_train/
    !tar -C $DATA_DIR -czf $DATA_DIR/cloud_folders/data/purpose_built_models_ocrnet_val/test.tar.gz test
    !cp $DATA_DIR/character_list $DATA_DIR/cloud_folders/data/purpose_built_models_ocrnet_val/
elif model_name in ("optical_inspection", "visual_changenet_classify"):
    !mkdir -p $DATA_DIR/cloud_folders/data/purpose_built_models_optical_inspection_train $DATA_DIR/cloud_folders/data/purpose_built_models_optical_inspection_val $DATA_DIR/cloud_folders/data/purpose_built_models_optical_inspection_test
    !tar -C $DATA_DIR/train -czf $DATA_DIR/cloud_folders/data/purpose_built_models_optical_inspection_train/images.tar.gz images
    !cp $DATA_DIR/train/dataset.csv $DATA_DIR/cloud_folders/data/purpose_built_models_optical_inspection_train/
    !tar -C $DATA_DIR/val -czf $DATA_DIR/cloud_folders/data/purpose_built_models_optical_inspection_val/images.tar.gz images
    !cp $DATA_DIR/val/dataset.csv $DATA_DIR/cloud_folders/data/purpose_built_models_optical_inspection_val/
    !tar -C $DATA_DIR/test -czf $DATA_DIR/cloud_folders/data/purpose_built_models_optical_inspection_test/images.tar.gz images
    !cp $DATA_DIR/test/dataset.csv $DATA_DIR/cloud_folders/data/purpose_built_models_optical_inspection_test/
elif model_name == "visual_changenet_segment":
    !mkdir -p $DATA_DIR/cloud_folders/data/purpose_built_models_visual_changenet_segment_train
    !tar -C $DATA_DIR/LEVIR-CD256 -czf $DATA_DIR/cloud_folders/data/purpose_built_models_visual_changenet_segment_train/A.tar.gz A
    !tar -C $DATA_DIR/LEVIR-CD256 -czf $DATA_DIR/cloud_folders/data/purpose_built_models_visual_changenet_segment_train/B.tar.gz B
    !tar -C $DATA_DIR/LEVIR-CD256 -czf $DATA_DIR/cloud_folders/data/purpose_built_models_visual_changenet_segment_train/list.tar.gz list
    !tar -C $DATA_DIR/LEVIR-CD256 -czf $DATA_DIR/cloud_folders/data/purpose_built_models_visual_changenet_segment_train/label.tar.gz label
elif model_name == "pointpillars":
    !mkdir -p $DATA_DIR/cloud_folders/data/purpose_built_models_pointpillars_train
    !tar -C $DATA_DIR -czf $DATA_DIR/cloud_folders/data/purpose_built_models_pointpillars_train/train.tar.gz train
    !tar -C $DATA_DIR -czf $DATA_DIR/cloud_folders/data/purpose_built_models_pointpillars_train/val.tar.gz val
elif model_name == "re_identification":
    !mkdir -p $DATA_DIR/cloud_folders/data/purpose_built_models_re_identification_train
    !tar -C $DATA_DIR/market1501 -czf $DATA_DIR/cloud_folders/data/purpose_built_models_re_identification_train/sample_train.tar.gz sample_train
    !tar -C $DATA_DIR/market1501 -czf $DATA_DIR/cloud_folders/data/purpose_built_models_re_identification_train/sample_test.tar.gz sample_test
    !tar -C $DATA_DIR/market1501 -czf $DATA_DIR/cloud_folders/data/purpose_built_models_re_identification_train/sample_query.tar.gz sample_query
elif model_name == 'centerpose':
    !mkdir -p $DATA_DIR/cloud_folders/data/purpose_built_models_centerpose_train
    !tar -C $DATA_DIR/{testing_categories} -czf $DATA_DIR/cloud_folders/data/purpose_built_models_centerpose_train/train.tar.gz train
    !tar -C $DATA_DIR/{testing_categories} -czf $DATA_DIR/cloud_folders/data/purpose_built_models_centerpose_train/val.tar.gz val
    !tar -C $DATA_DIR/{testing_categories} -czf $DATA_DIR/cloud_folders/data/purpose_built_models_centerpose_train/test.tar.gz test

### Final step: Upload the tar files to your cloud storage and move on to running the API requests example notebooks

In [ ]:
!python3 -m pip install --upgrade awscli
ACCESS_KEY=FIXME4.1
SECRET_KEY=FIXME4.2
BUCKET_NAME=FIXME4.3

if model_name == "action_recognition":
    !AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/cloud_folders/data/purpose_built_models_action_recognition_train s3://{BUCKET_NAME}/data/purpose_built_models_action_recognition_train/$model_type --recursive

elif model_name == "ml_recog":
    !AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes s3://{BUCKET_NAME}/data/purpose_built_models_ml_recog_train/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes --recursive
    !AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes s3://{BUCKET_NAME}/data/purpose_built_models_ml_recog_train/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes --recursive

elif model_name == "ocdnet":
    !AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/cloud_folders/data/purpose_built_models_ocdnet_train s3://{BUCKET_NAME}/data/purpose_built_models_ocdnet_train --recursive
    !AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/cloud_folders/data/purpose_built_models_ocdnet_val s3://{BUCKET_NAME}/data/purpose_built_models_ocdnet_val --recursive

elif model_name == "ocrnet":
    !AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/cloud_folders/data/purpose_built_models_ocrnet_train s3://{BUCKET_NAME}/data/purpose_built_models_ocrnet_train --recursive
    !AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/cloud_folders/data/purpose_built_models_ocrnet_val s3://{BUCKET_NAME}/data/purpose_built_models_ocrnet_val --recursive

elif model_name in ("optical_inspection", "visual_changenet_classify"):
    !AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/cloud_folders/data/purpose_built_models_optical_inspection_train s3://{BUCKET_NAME}/data/purpose_built_models_optical_inspection_train --recursive
    !AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/cloud_folders/data/purpose_built_models_optical_inspection_val s3://{BUCKET_NAME}/data/purpose_built_models_optical_inspection_val --recursive
    !AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/cloud_folders/data/purpose_built_models_optical_inspection_test s3://{BUCKET_NAME}/data/purpose_built_models_optical_inspection_test --recursive

elif model_name == "visual_changenet_segment":
    !AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/cloud_folders/data/purpose_built_models_visual_changenet_segment_train s3://{BUCKET_NAME}/data/purpose_built_models_visual_changenet_segment_train --recursive

elif model_name == "pointpillars":
    !AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/cloud_folders/data/purpose_built_models_pointpillars_train s3://{BUCKET_NAME}/data/purpose_built_models_pointpillars_train --recursive

elif model_name == "pose_classification":
    !AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/$model_type s3://{BUCKET_NAME}/data/purpose_built_models_pose_classification_train/$model_type --recursive

elif model_name == "re_identification":
    !AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/cloud_folders/data/purpose_built_models_re_identification_train s3://{BUCKET_NAME}/data/purpose_built_models_re_identification_train --recursive

elif model_name == 'centerpose':
    !AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/cloud_folders/data/purpose_built_models_centerpose_train s3://{BUCKET_NAME}/data/purpose_built_models_centerpose_train --recursive

In [ ]:
# This will be the paths in your API/TAO-CLIENT Notebooks
train_dataset_path = f"/data/purpose_built_models_{model_name}_train" # /data/purpose_built_models_{model_name}_train/$model_type for action_recognition 
eval_dataset_path = f"/data/purpose_built_models_{model_name}_val"  # ocdnet, ocrnet, optical_inspection, visual_changenet_classify
test_dataset_path = f"/data/purpose_built_models_{model_name}_test"  # optical_inspection, visual_changenet_classify